In [ ]:
# import relevant libraries
import networkx as nx
import matplotlib.pyplot as plt
from operator import itemgetter
import wikipedia

# Specify name of the starting pages
seed1 = "Aqualung".title()
seed2 = "Ian Anderson".title()


# Keep track of pages to process; we'll only do 2 layers,
# process links as a BFS
toDo_lst1 = [(0, seed1)]
toDo_lst2 = [(0, seed2)]

toDo_set1 = set(seed1)
toDo_set2 = set(seed2)

done_set1 = set()
done_set2 = set()

# Create two directed graphs
F = nx.DiGraph()
G = nx.DiGraph()

layer1, page1 = toDo_lst1[0]
layer2, page2 = toDo_lst2[0]
    
    
count1 = 0

while layer1 < 2:
    del toDo_lst1[0]
    done1 = done_set1.add(page1)
    print(layer1, page1)
    count1 = count1 + 1
    if (count1 > 50):
        break
    try:
            # Download the selected page
        wiki1 = wikipedia.page(page1)
    except:
        layer1, page1 = toDo_lst1[0]
        print("Could not load", page1)
        continue

    
    for link in wiki1.links:
        link = link.title()
        if link not in toDo_set1 and link not in done_set1 and len(done_set1) < 50:
            toDo_lst1.append((layer1+1, link))
            toDo_set1.add(link)
        F.add_edge(page1, link)

    layer1, page1 = toDo_lst1[0]


count2 = 0

while layer2 < 2:
    del toDo_lst2[0]
    done2 = done_set2.add(page2)
    print(layer2, page2)
    count2 = count2 + 1
    if (count2 > 50):
        break
    try:
            # Download the selected page
        wiki2 = wikipedia.page(page2)
    except:
        layer2, page2 = toDo_lst2[0]
        print("Could not load", page2)
        continue

    for link in wiki2.links:
        link = link.title()
        if link not in toDo_set2 and link not in done_set2 and len(done_set2) < 50:
            toDo_lst2.append((layer2+1, link))
            toDo_set2.add(link)
        G.add_edge(page2, link)

    layer2, page2 = toDo_lst2[0]

print("The Aqualung graph has {} nodes, {} edges". format(len(F),nx.number_of_edges(F)))
print("The Ian Anderson graph has {} nodes, {} edges". format(len(G),nx.number_of_edges(G)))


# the following will get rid of some of "self loops" in both graphs
F.remove_edges_from(nx.selfloop_edges(F))
G.remove_edges_from(nx.selfloop_edges(G))

# F and G are both gigantic graphs. I make smaller subgraphs by only including nodes that have degree >= 2
core1 = [node for node, deg in dict(F.degree()).items() if deg >= 2]
H = nx.subgraph(F, core1)

core2 = [node for node, deg in dict(G.degree()).items() if deg >= 2]
I = nx.subgraph(G, core2)


print("The new Aqualung graph has {} nodes, {} edges".format(len(H), nx.number_of_edges(H)))
print("The new Ian Anderson graph has {} nodes, {} edges".format(len(I), nx.number_of_edges(I)))

# Display the smaller graph for Aqualung
pos = nx.spring_layout(H)
plt.figure(figsize=(100,100))
nx.draw_networkx(H, pos=pos, with_labels=True)
plt.axis('off')
plt.show()

# Display the smaller graph for Ian Anderson
pos = nx.spring_layout(I)
plt.figure(figsize=(100,100))
nx.draw_networkx(I, pos=pos, with_labels=True)
plt.axis('off')
plt.show()

# Display a list of subjects sorted by in-degree
top_indegree = sorted(dict(G.in_degree()).items(),
                      reverse=True, key=itemgetter(1))[:100]
print("\n".join(map(lambda t: "{} {}".format(*reversed(t)), top_indegree)))

In [ ]:
# import relevant library
from numpy import array
from itertools import product
import math

In [ ]:
def simrank_similarity(G, source=None, target=None, importance_factor=0.9, max_iterations=100, tolerance=1e-4):  
    prevsim = None  
    newsim = {u: {v: 1 if u == v else 0 for v in G} for u in G}  
    avg_sim = lambda s: sum(newsim[w][x] for (w, x) in s) / len(s) if s else 0.0  
    sim = lambda u, v: importance_factor * avg_sim(list(product(G[u], G[v]))) 
    
    for _ in range(max_iterations):    
        if prevsim and _is_close(prevsim, newsim, tolerance):      
            break    
        prevsim = newsim    
        newsim = {u: {v: sim(u, v) if u is not v else 1 for v in newsim[u]} for u in newsim}  
    
    if source is not None and target is not None:    
        return newsim[source][target]  
    if source is not None:    
        return newsim[source]  
    return newsim
        
def _is_close(d1, d2, atolerance=0, rtolerance=0): 
    if not isinstance(d1, dict) and not isinstance(d2, dict):
        return abs(d1 - d2) <= atolerance + rtolerance * abs(d2)  
    return all(all(_is_close(d1[u][v], d2[u][v]) for v in d1[u]) for u in d1)

In [ ]:
# for the aqualung graph
sim1 = simrank_similarity(H)
sim_nodes1 = [[sim1[u][v] for v in sorted(sim1[u])] for u in sorted(sim1)]
sim_array1 = array(sim_nodes1)

nodeList1 = list(H.nodes)
n1 = len(nodeList1)
threshold1 = 0.01
for i in range(n1):
    for j in range(n1):
        if ((sim_array1[i][j] >= threshold1) and (i != j)):
            print("(",nodeList1[i], ",", nodeList1[j], '): ', sim_array1[i][j])

In [ ]:
# for the Ian Anderson graph
sim2 = simrank_similarity(I)
sim_nodes2 = [[sim2[u][v] for v in sorted(sim2[u])] for u in sorted(sim2)]
sim_array2 = array(sim_nodes2)

nodeList2 = list(I.nodes)
n2 = len(nodeList2)
threshold2 = 0.01
for i in range(n2):
    for j in range(n2):
        if ((sim_array2[i][j] >= threshold2) and (i != j)):
            print("(",nodeList2[i], ",", nodeList2[j], '): ', sim_array2[i][j])

In [ ]:
J = H.copy()

J.remove_nodes_from(n for n in H if n in I)
# K = nx.difference(J, I)

remove1 = [node for node,degree in dict(J.degree()).items() if degree == 0]
J.remove_nodes_from(remove1)
print(nx.info(J))

In [ ]:
L = H.copy()
L.remove_nodes_from(n for n in H if n not in I)

remove2 = [node for node,degree in dict(L.degree()).items() if degree == 0]
L.remove_nodes_from(remove2)
print(nx.info(L))